<a href="#1-Definition" style="margin-left: 0px;">1 Definition</a>   
<a href="#2-XGBoost,-LightGBM,-CatBoos" style="margin-left: 0px;">2 XGBoost, LightGBM, CatBoos</a>  
<a href="#3-Implementation" style="margin-left: 0px;">3 Implementation</a>  

### 1 Definition

Градиентный бустинг — это метод ансамблевого обучения, который строит модель последовательно, добавляя слабые модели (обычно решающие деревья), каждая из которых обучается на остатках ошибок предыдущих моделей, чтобы постепенно улучшать точность предсказаний.

### 2 XGBoost, LightGBM, CatBoos

### Есть 3 библиотеки, в которых реализован Градиентный бустинг

| Характеристика                  | XGBoost                        | LightGBM                             | CatBoost                            | sklearn.GradientBoosting           |
|----------------------------------|---------------------------------|--------------------------------------|-------------------------------------|-----------------------------------|
| 🚀 Скорость обучения            | Средняя                         | Очень высокая                        | Средняя                             | Средняя                           |
| 🧠 Обработка категорий          | ❌ Нет (нужно вручную)          | ⚠️ Частичная (требует `category`)    | ✅ Да (автоматически)               | ❌ Нет (нужно вручную)             |
| 🌿 Рост дерева                  | Level-wise (по слоям)           | Leaf-wise (по наилучшему листу)      | Oblivious Trees (симметричные)      | Level-wise (по слоям)             |
| 📈 Качество без настройки       | Хорошее                         | Хорошее                              | Отличное                            | Хорошее                           |
| 🔧 Настраиваемость              | Очень гибкий                    | Гибкий                               | Меньше параметров                   | Средняя                           |
| 🧩 Работа с NaN                 | ✅ Да                           | ✅ Да                                 | ✅ Да                                | ✅ Да                             |
| 🧮 Регуляризация               | ✅ L1 и L2                      | ✅ L1 и L2                           | ✅ Только L2                        | ❌ Почти нет прямой регуляризации |
| ⚙️ Поддержка GPU               | ✅ Да                           | ✅ Да                                 | ✅ Да                                | ❌ Нет                            |


| Параметр                         | XGBoost                      | LightGBM                     | CatBoost                     | sklearn.GradientBoosting           |
|-----------------------------------|------------------------------|-----------------------------|-----------------------------|-----------------------------------|
| **Регуляризация L1 (Lasso)**      | ✅ Есть (через `reg_alpha`)   | ✅ Есть (через `lambda_l1`)   | ❌ Нет (только L2)            | ❌ Нет (только через параметры дерева) |
| **Регуляризация L2 (Ridge)**      | ✅ Есть (через `reg_lambda`)  | ✅ Есть (через `lambda_l2`)   | ✅ Есть (через `l2_leaf_reg`) | ❌ Почти нет                     |
| **Тип бустинга (Booster)**        | ✅ Есть (например, `dart`)    | ❌ Нет                        | ❌ Нет                        | ❌ Нет                            |
| **Раннее завершение**             | ✅ Есть (`early_stopping`)    | ✅ Есть                       | ✅ Есть                       | ✅ Есть                           |
| **Гибкость контроля роста дерева**| ✅ Очень высокая (параметры `max_depth`, `min_child_weight`, и другие) | ✅ Высокая (но немного меньше параметров) | ✅ Высокая, но с фиксированными параметрами | Средняя                           |


### 2.1 XGBoost (Extreme Gradient Boosting)   

**Смысл:**  
Улучшение базового градиентного бустинга с акцентом на производительность, оптимизацию и гибкость.

**Особенности:**  

#### 1) Очень гибкий, с множеством параметров

**Плюсы:**  

Хорошо подходит для структурированных данных (таблицы).  
Высокая производительность при больших объёмах данных.  
Много возможностей для настройки.  

**Минусы:**  

Менее эффективно работает с категориальными признаками (нужен Label Encoding или One-Hot Encoding).  
Требует ручной настройки гиперпараметров для оптимальных результатов.  

**Режим DART**  
Режим DART (Dropouts meet Multiple Additive Regression Trees) в библиотеке XGBoost — это техника регуляризации, которая помогает избежать переобучения за счёт применения метода "dropout" к деревьям. Этот метод был вдохновлён техникой dropout, используемой в нейронных сетях, но адаптирован для градиентного бустинга.  

При обучении ансамбля деревьев в XGBoost, DART случайным образом отключает (drop) некоторые деревья из ансамбля на каждой итерации. Таким образом, модель избегает чрезмерного подгонки к обучающим данным, так как не все деревья участвуют в предсказаниях в каждый момент времени.

DART делает акцент на два аспекта:

Случайное исключение деревьев на каждой итерации.
Компенсация весов оставшихся деревьев, чтобы поддерживать корректный вклад в предсказания.

### Пример с компенсацией (Один из способов):  

### ❓ Что будет, если второе дерево (дающее +2) исключается?

Если мы пропускаем второе дерево, то его вклад $0.1 \cdot 2 = 0.2$ **не добавляется**.

---

#### 📉 Без компенсации:

$$
\text{pred} = 0.7 + 0.1 \cdot 0.5 = 0.75
$$

Мы просто потеряли $+0.2$. Было бы $0.95$, а стало **$0.75$**.

---

#### 🔁 С компенсацией (как ты предлагаешь):

Допустим, XGBoost (или другой бустинг) **усиливает оставшиеся деревья**, чтобы компенсировать потерю.

- Пропущено: $+0.2$ (из-за исключения дерева с выходом +2).
- Единственное оставшееся дерево даёт: $+0.5$
- Его вклад без компенсации: $0.1 \cdot 0.5 = 0.05$

Чтобы компенсировать потерю $0.2$, нужно увеличить вес оставшегося дерева.

---

#### 🧮 Формула компенсации:

$$
\text{compensation_factor} = \frac{0.2}{0.05} = 4
$$

То есть, оставшееся дерево должно быть усилено **в 4 раза**, чтобы покрыть потерю.

---

#### ✅ Итоговое предсказание:

$$
\text{pred} = 0.7 + 4 \cdot (0.1 \cdot 0.5) = 0.7 + 0.2 = 0.9
$$


### 2.2 LightGBM (Light Gradient Boosting Machine)  

LightGBM (Leaf-wise): Как "умный студент" — учится быстро, но может перестараться.  

XGBoost (Level-wise): Как "методичный профессор" — учится медленнее, но стабильнее.  

**Смысл:** Оптимизация для больших и высокоразмерных наборов данных. LightGBM фокусируется на скорости и памяти.  

**Особенности:**  

#### 1) Умное построение деревьев:  
Использует подход Leaf-wise growth (рост дерева по наиболее полезной ветке, а не по всем сразу, как в XGBoost). Это ускоряет обучение, но повышает риск переобучения.    

**Level-wise:** Представь, что ты растишь дерево строго по слоям — на каждом уровне появляются все возможные разветвления.

**Leaf-wise:** А тут — дерево растёт вглубь только там, где ошибка больше, т.е. ты "вылизываешь" самые важные участки данных. Дерево получается не симметричным


#### 2) Поддержка категориальных признаков:  
Автоматическая обработка категорий (через специальный метод биннинга).    

#### 3) Gradient-based One-Side Sampling (GOSS):
Для ускорения работы модель выбирает подмножество данных, игнорируя менее значимые для текущей итерации объекты. 

#### 🔍 Что делает GOSS:
Сначала вычисляются градиенты для всех объектов (то есть — насколько сильно на каждом объекте модель ошиблась).

Сохраняем только самые важные объекты:

Выбираем все строки, где ошибка большая (т.е. градиенты большие) — их нельзя терять.

А среди тех, где ошибка маленькая — случайно берём часть, чтобы ускорить обучение.

Чтобы не потерять баланс — модель потом добавляет вес к тем случайно выбранным строкам, чтобы они не стали «менее важными» только из-за подвыборки.

**Exclusive Feature Bundling (EFB):**  
Объединение малозначимых признаков в один, чтобы уменьшить размерность данных.  
Параллелизм и распределённые вычисления.  

**Плюсы:**  

Быстрее, чем XGBoost, при большом количестве признаков и строк.  
Эффективно использует память.  
Хорошо работает на больших данных.  

**Минусы:**  

Leaf-wise growth может привести к переобучению, особенно при недостатке данных.   
Меньше возможностей для тонкой настройки, чем в XGBoost.  

### 2.3 CatBoost (Categorical Boosting)  

**Смысл:** Упрощение работы с категориальными данными и улучшение производительности на небольших и средних наборах данных.  
**Особенности:**  

**Категориальные признаки:**
Поддерживает категориальные данные "из коробки", используя целевую статистику и методы устранения утечек данных (без необходимости в One-Hot Encoding).  
**Устойчивость к переобучению:**  
Использует порядок данных (Ordered Boosting), чтобы предотвратить утечку информации между обучением и тестированием.

**Эффективность на малых данных:**  
Хорошо работает на данных с ограниченным числом строк.
**Автоматизация:**  
Меньше потребности в ручной настройке гиперпараметров.
**Поддержка текстовых и временных данных:**  
CatBoost включает встроенные методы обработки текстовых данных.

**Плюсы:**  

Простая работа с категориальными признаками.  
Отличная обобщающая способность.  
Лучшая производительность на задачах с малым количеством данных.  
Меньше усилий для настройки модели.  

**Минусы:**

Может быть медленнее, чем LightGBM и XGBoost на больших наборах данных.  
Требует больше оперативной памяти при обработке категориальных признаков.  

#### Обработка категориальных признаков в CatBoost

CatBoost действительно использует **целевое кодирование (Target Encoding)** с модификацией **Ordered Target Encoding**, которая учитывает порядок наблюдений (это важно для борьбы с переобучением).

#### Leave-One-Out (LOO) в CatBoost

#### Основная идея:
- Категориальный признак заменяется на среднее значение целевой переменной для данной категории.
- **Но при кодировании текущей строки она исключается из расчёта**, чтобы избежать утечки данных (data leakage).
Когда мы кодируем категорию в конкретной строке, её значение цели (y) не участвует в расчёте среднего для этой же категории.  
Зачем? Чтобы модель не "подглядывала" в правильный ответ (это и есть утечка данных)  

#### На Примере:

#### 1) Использует LOO-кодирование (исключаем текущую строку)

#### 2) Ordered Target Encoding (не просто LOO):
- CatBoost использует **порядковый подход** (на основе "времени" — порядка наблюдений), чтобы избежать переобучения.
- При кодировании категории в текущей строке используются **только предыдущие наблюдения** (как во временных рядах).


### Классификация

| Порядок | Категория  | Таргет | Закодированное значение (примерно)                        |
|---------|------------|--------|-----------------------------------------------------------|
| 1       | "красный"  | 1      | Начальное приближение по "красному" (например, 0.5)       |
| 2       | "синий"    | 0      | Начальное приближение по "синему" (например, 0.5)         |
| 3       | "красный"  | 0      | (1) / 1 = 1.0                                              |
| 4       | "красный"  | 1      | (1 + 0) / 2 = 0.5                                          |

### Регрессия

| Порядок | Категория  | Таргет | Закодированное значение (расчёт)                                               |
|---------|------------|--------|---------------------------------------------------------------------------------|
| 1       | "красный"  | 1      | Нет предыдущих → 2.5 (начальное)                                               |
| 2       | "синий"    | 2      | Нет предыдущих → 2.5 (начальное)                                               |
| 3       | "красный"  | 3      | Предыдущий "красный" (строка 1, таргет=1) → (1) / 1 = 1.0                      |
| 4       | "красный"  | 4      | Предыдущие "красные" (строки 1 и 3: таргеты=1, 3) → (1 + 3) / 2 = 2.0          |


#### Дополнительные механизмы:
- Добавляется **шум (random noise)** или **prior** (среднее по всем данным), чтобы снизить дисперсию.
- Учитывается **частота категорий** (редкие категории сильнее сглаживаются).

In [1]:
import pandas as pd
import numpy as np
import copy

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from collections import Counter
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
# import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler

C:\Users\Rinat\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### 3 Implementation

#### Classification

| №  | Название колонки | Тип данных       | Описание |
|----|------------------|------------------|----------|
| 1  | `PassengerId`    | `int`            | Уникальный ID пассажира |
| 2  | `Survived`       | `int` (0/1)      | Выжил (1) или нет (0) — **таргет для предсказания** |
| 3  | `Pclass`         | `int` (1-3)      | Класс билета:<br>• 1 = 1-й класс<br>• 2 = 2-й класс<br>• 3 = 3-й класс |
| 4  | `Name`           | `string`         | Имя пассажира (включая титул, например, "Mr.", "Miss") |
| 5  | `Sex`            | `string`         | Пол (`"male"` или `"female"`) |
| 6  | `Age`            | `float`          | Возраст (есть пропуски — `NaN`) |
| 7  | `SibSp`          | `int`            | Количество siblings (братья/сёстры) + spouses (супруги) на борту |
| 8  | `Parch`          | `int`            | Количество parents (родители) + children (дети) на борту |
| 9  | `Ticket`         | `string`         | Номер билета (может содержать буквы и цифры) |
| 10 | `Fare`           | `float`          | Стоимость билета |
| 11 | `Cabin`          | `string`         | Номер каюты (много пропусков — `NaN`) |
| 12 | `Embarked`       | `string`         | Порт посадки:<br>• `C` = Cherbourg<br>• `Q` = Queenstown<br>• `S` = Southampton |

In [2]:
data = pd.read_csv(r"../00 Data/titanic.csv")

print(data.shape)
data.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# Замена пропусков в Age медианным значением
data['Age'] = data['Age'].fillna(data['Age'].median())

# Замена пропусков в Embarked модой
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

# Семейный размер
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1

# Одиночка (1, если FamilySize = 1)
data['IsAlone'] = (data['FamilySize'] == 1).astype(int)

# Encoder
encoder = TargetEncoder(cols=['Sex', 'Embarked', 'FamilySize'])

data[['Sex', 'Embarked', 'FamilySize']] = encoder.fit_transform(
    data[['Sex', 'Embarked', 'FamilySize']], 
    data['Survived']  # Целевая переменная
)

In [4]:
# StandardScaler для всех признаков
features = data.columns.drop('Survived')
data[features] = StandardScaler().fit_transform(data[features])

In [5]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,0,0.827377,-0.737695,-0.565736,0.432793,-0.473674,-0.502445,-0.539973,1.290322,-1.231645
1,1,-1.566107,1.355574,0.663861,0.432793,-0.473674,0.786845,2.044556,1.290322,-1.231645
2,1,0.827377,1.355574,-0.258337,-0.474545,-0.473674,-0.488854,-0.539973,-0.687981,0.811922
3,1,-1.566107,1.355574,0.433312,0.432793,-0.473674,0.420730,-0.539973,1.290322,-1.231645
4,0,0.827377,-0.737695,0.433312,-0.474545,-0.473674,-0.486337,-0.539973,-0.687981,0.811922


### Пример: 

| Возраст | Зарплата | Купил (y) |
|---------|----------|-----------|
| 25      | 40 000   | 0         |
| 30      | 60 000   | 0         |
| 35      | 80 000   | 1         |
| 40      | 100 000  | 1         |
| 45      | 120 000  | 1         |

❗❗❗ **В градиентном бустинге для классификации каждое дерево — это дерево регрессии, даже если исходная задача — классификация. То есть таргеты это вещественные чила! Это ключевая особенность метода!**  

**Шаг 0: Инициализация:**

Вычисляем начальное предсказание (логарифм шансов):

Среднее y: mean_y = (0+0+1+1+1)/5 = 0.6

initial_pred = log(0.6 / (1-0.6)) ≈ 0.405

Начальные предсказания для всех объектов: [0.405, 0.405, 0.405, 0.405, 0.405]

**Переводим в вероятности (сигмоида):**  
proba = 1 / (1 + exp(-0.405)) ≈ [0.6, 0.6, 0.6, 0.6, 0.6]

**🌳 Дерево 1: Предсказываем первые градиенты**

**Шаг 1.1: Вычисляем градиенты (псевдо-остатки)**  

Текущие вероятности:   
pred = [0.6, 0.6, 0.6, 0.6, 0.6]    

Градиенты (y - p):    
gradients = [0-0.6, 0-0.6, 1-0.6, 1-0.6, 1-0.6] = [-0.6, -0.6, 0.4, 0.4, 0.4]   
❗ Это и есть новый таргет дерева  

**Шаг 1.2: Строим дерево**       
tree.fit(X[:, feature_subset], gradients)  

**Шаг 1.3: Считаем новое предсказание:**   
pred += learning_rate * tree.predict(X)  # Корректируем предыдущие предсказания   
pred = [0.6, 0.6, 0.6, 0.6, 0.6] + 0.1 * [-0.6, -0.6, 0.4, 0.4, 0.4]   
pred = [0.54, 0.54, 0.64, 0.64, 0.64]  # Новые предсказания!  

Для y=0 предсказания уменьшились (0.6 → 0.54)  

Для y=1 увеличились (0.6 → 0.64)

**🌳 Дерево 2: Корректируем остатки**

**Шаг 2.1: Вычисляем новые градиенты**  
Текущие вероятности:  
pred = [0.54, 0.54, 0.64, 0.64, 0.64]  

Градиенты (y - p):  
gradients = [0-0.54, 0-0.54, 1-0.64, 1-0.64, 1-0.64] = [-0.54, -0.54, 0.36, 0.36, 0.36]  
❗ Это и есть новый таргет дерева  

**Шаг 2.2: Строим дерево**       
tree.fit(X[:, feature_subset], gradients)    

**Шаг 2.3: Считаем новое предсказание:**    
pred += learning_rate * tree.predict(X)  # Корректируем предыдущие предсказания   
pred = [0.54, 0.54, 0.64, 0.64, 0.64] + 0.1 * [-0.54, -0.54, 0.36, 0.36, 0.36]   
pred = [0.486, 0.486, 0.676, 0.676, 0.676]  # Ещё лучше!    

Для y=0: 0.54 → 0.486 (ещё ближе к 0)  

Для y=1: 0.64 → 0.676 (ещё ближе к 1)  

**Основные критерии остановки**

| Критерий               | Пример значения | Что означает                                        |
|------------------------|-----------------|-----------------------------------------------------|
| n_estimators           | 100             | Максимальное количество деревьев                    |
| early_stopping_rounds  | 10              | Остановка, если ошибка не уменьшается 10 итераций подряд |
| min_impurity_decrease  | 0.0             | Минимальное улучшение качества разделения           |


In [6]:
class My_GDBTClassifier:
    def __init__(self, n_estimators=100, max_features='sqrt', max_depth=3, learning_rate=0.1, random_state=42):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.random_state = random_state
        self.trees = []
        self.feature_indices = []
        self.initial_pred = None
    
    def _calculate_gradients(self, y, pred):
        return y - pred
    
    def _get_feature_subset(self, n_features):
        if self.max_features == 'sqrt':
            n_subset = int(np.sqrt(n_features))
        elif self.max_features == 'log2':
            n_subset = int(np.log2(n_features))
        elif isinstance(self.max_features, int):
            n_subset = self.max_features
        else:
            raise ValueError("Invalid value for max_features")
        
        rng = np.random.RandomState(self.random_state)
        return rng.choice(n_features, size=n_subset, replace=False)
    
    def fit(self, X, y):
        eps = 1e-10
        mean_y = np.clip(np.mean(y), eps, 1-eps)
        self.initial_pred = np.log(mean_y / (1 - mean_y))
        pred = np.full_like(y, self.initial_pred, dtype=float)

        for _ in range(self.n_estimators):
            proba = 1 / (1 + np.exp(-pred))
            gradients = y - proba

            feature_subset = self._get_feature_subset(X.shape[1])
            self.feature_indices.append(feature_subset)

            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X[:, feature_subset], gradients)
            self.trees.append(tree)

            pred += self.learning_rate * tree.predict(X[:, feature_subset])
    
    def predict(self, X):
        pred = np.full(X.shape[0], self.initial_pred)
        
        for tree, features in zip(self.trees, self.feature_indices):
            pred += self.learning_rate * tree.predict(X[:, features])
        
        return (1 / (1 + np.exp(-pred))) > 0.5  # Возвращаем бинарные предсказания
    
    def predict_proba(self, X):
        pred = np.full(X.shape[0], self.initial_pred)
        
        for tree, features in zip(self.trees, self.feature_indices):
            pred += self.learning_rate * tree.predict(X[:, features])
        
        proba = np.zeros((X.shape[0], 2))
        proba[:, 1] = 1 / (1 + np.exp(-pred))  # Вероятность класса 1
        proba[:, 0] = 1 - proba[:, 1]  # Вероятность класса 0
        return proba

In [7]:
max_features = data.iloc[:,1:].values.shape[1]

In [8]:
%%time
# Обучаем модель дерева решений
my_gdbt = My_GDBTClassifier(n_estimators=5, max_depth=15, max_features=max_features, random_state=42)
my_gdbt.fit(data.iloc[:,1:].values, data.iloc[:,0].values)

CPU times: total: 31.2 ms
Wall time: 29.2 ms


In [9]:
%%time
my_y_proba = my_gdbt.predict_proba(data.iloc[:,1:].values)[:,1]

CPU times: total: 31.2 ms
Wall time: 6.99 ms


In [10]:
%%time

# Обучаем модель градиентного бустинга из Scikit-learn
sklearn_gbdt = GradientBoostingClassifier(n_estimators=5, 
                                        max_depth=15, 
                                        max_features=max_features, 
                                        random_state=42)
sklearn_gbdt.fit(data.iloc[:,1:].values, data.iloc[:,0].values)

CPU times: total: 109 ms
Wall time: 95.2 ms


GradientBoostingClassifier(max_depth=15, max_features=9, n_estimators=5,
                           random_state=42)

In [11]:
%%time
y_proba = sklearn_gbdt.predict_proba(data.iloc[:,1:].values)[:,1]

CPU times: total: 0 ns
Wall time: 5 ms


In [12]:
# Вычисление AUC-ROC
AUC_my = roc_auc_score(data['Survived'].values, my_y_proba)
AUC_sclearn = roc_auc_score(data['Survived'].values, y_proba)

print(f"AUC-ROC (Custom Model): {AUC_my:.4f}")
print(f"AUC-ROC (Sklearn Model): {AUC_sclearn:.4f}")

AUC-ROC (Custom Model): 0.9959
AUC-ROC (Sklearn Model): 0.9974


#### Результат схожий

#### Regression

In [13]:
data = pd.read_csv(r"../00 Data/real estate.csv")

In [14]:
print(data.shape)
data.head()

(49352, 24)


,bathrooms,bedrooms,interest_level,price,Elevator,CatsAllowed,HardwoodFloors,DogsAllowed,Doorman,Dishwasher,...,LaundryinUnit,RoofDeck,OutdoorSpace,DiningRoom,HighSpeedInternet,Balcony,SwimmingPool,LaundryInBuilding,NewConstruction,Terrace
0,1.0,1,1,2400,0,1,1,1,0,1,...,0,0,0,1,0,0,0,0,0,0
1,1.0,2,0,3800,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1.0,2,1,3495,1,0,1,0,1,1,...,1,0,0,0,0,0,0,0,0,0
3,1.5,3,1,3000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1.0,0,0,2795,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
X = data.drop(['price'],axis=1).to_numpy()
y = data['price'].to_numpy()

**Пример:**   

| Возраст | Зарплата | Работает (годы) (y) |
|---------|----------|---------------------|
| 25      | 40 000   | 1                   |
| 30      | 60 000   | 3                   |
| 35      | 80 000   | 5                   |
| 40      | 100 000  | 7                   |
| 45      | 120 000  | 9                   |


**Шаг 0: Инициализация**  

Начальное предсказание — среднее значение y:  
mean_y = (1+3+5+7+9)/5 = 5.0  
pred = [5.0, 5.0, 5.0, 5.0, 5.0]  

**🌳 Дерево 1: Предсказываем первые остатки**  

**Шаг 1.1: Вычисляем остатки (градиенты для MSE)**  

gradients = y - pred = [1-5, 3-5, 5-5, 7-5, 9-5] = [-4, -2, 0, 2, 4]

**Шаг 1.2: Строим дерево**  
tree.fit(X[:, feature_subset], gradients)

**Шаг 1.3: Считаем новое предсказание:**  

pred += learning_rate * tree.predict(X) # Корректируем предыдущие предсказания  
Допустим, выбрали признак "Зарплата":    
pred = [5.0, 5.0, 5.0, 5.0, 5.0] + 0.1 * [-3.0, -3.0, 3.0, 3.0, 3.0]  
pred = [4.7, 4.7, 5.3, 5.3, 5.3]  


**🌳 Дерево 2: Корректируем остатки**  

**Шаг 2.1: Новые остатки**  
gradients = [1-4.7, 3-4.7, 5-5.3, 7-5.3, 9-5.3] = [-3.7, -1.7, -0.3, 1.7, 3.7]  

**Шаг 2.2: Строим дерево**  
tree.fit(X[:, feature_subset], gradients)  

**Шаг 2.3: Считаем новое предсказание:**  

pred += learning_rate * tree.predict(X) # Корректируем предыдущие предсказания  
Допустим, выбрали признак "Зарплата":    
pred = [4.7, 4.7, 5.3, 5.3, 5.3] + 0.1 * [-2.7, -2.7, 1.7, 1.7, 1.7]  
pred = [4.43, 4.43, 5.47, 5.47, 5.47]   

In [16]:
class My_GDBTRegressor:
    def __init__(self, n_estimators=100, max_features='sqrt', max_depth=3, learning_rate=0.1, random_state=42):
        self.n_estimators = n_estimators  # Количество деревьев
        self.max_features = max_features  # Количество признаков для каждого дерева
        self.max_depth = max_depth        # Максимальная глубина дерева
        self.learning_rate = learning_rate # Коэффициент обучения
        self.random_state = random_state 
        self.trees = []                    # Список деревьев
        self.feature_indices = []          # Индексы признаков для каждого дерева
        self.initial_pred = None           # Начальное предсказание (среднее значение)

    def _get_feature_subset(self, n_features):
        if self.max_features == 'sqrt':
            n_subset = int(np.sqrt(n_features))
        elif self.max_features == 'log2':
            n_subset = int(np.log2(n_features))
        elif isinstance(self.max_features, int):
            n_subset = self.max_features
        else:
            raise ValueError("Invalid value for max_features")
        
        rng = np.random.RandomState(self.random_state)
        return rng.choice(n_features, size=n_subset, replace=False)

    def fit(self, X, y):
        # Инициализируем начальное предсказание (среднее значение)
        self.initial_pred = np.mean(y)
        pred = np.full_like(y, self.initial_pred, dtype=float)

        for _ in range(self.n_estimators):
            # Градиенты - это разница между истинными значениями и предсказаниями
            gradients = y - pred

            # Формируем подмножество признаков для дерева
            feature_subset = self._get_feature_subset(X.shape[1])
            self.feature_indices.append(feature_subset)

            # Обучаем дерево на основе градиентов
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X[:, feature_subset], gradients)
            self.trees.append(tree)

            # Обновляем предсказания
            pred += self.learning_rate * tree.predict(X[:, feature_subset])
    
    def predict(self, X):
        # Начинаем с начального предсказания
        pred = np.full(X.shape[0], self.initial_pred)

        # Добавляем предсказания от каждого дерева
        for tree, features in zip(self.trees, self.feature_indices):
            pred += self.learning_rate * tree.predict(X[:, features])
        
        return pred  # Возвращаем предсказанные значения

    def predict_proba(self, X):
        # Метод возвращает только вероятность для задачи регрессии (предсказания без порога)
        pred = np.full(X.shape[0], self.initial_pred)

        for tree, features in zip(self.trees, self.feature_indices):
            pred += self.learning_rate * tree.predict(X[:, features])

        return pred  # Для регрессии возвращаем сами предсказанные значения

In [17]:
%%time
# 1. Создание модели дерева решений
my_model_gdbt_regressor = My_GDBTRegressor(max_depth=5)

# 2. Обучение модели
my_model_gdbt_regressor.fit(X,y)

CPU times: total: 1.78 s
Wall time: 1.79 s


In [18]:
%%time
# 3. Получение предсказаний
my_y_pred = my_model_gdbt_regressor.predict(X)

CPU times: total: 328 ms
Wall time: 331 ms


In [19]:
%%time
# 1. Создание модели дерева решений
model_gdbt_regressor = GradientBoostingRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=42)

# 2. Обучение модели
model_gdbt_regressor.fit(X,y)

CPU times: total: 3.75 s
Wall time: 3.75 s


GradientBoostingRegressor(random_state=42)

In [20]:
# 3. Получение предсказаний
y_pred = model_gdbt_regressor.predict(X)

In [21]:
my_mse = mean_squared_error(y, my_y_pred)
mse = mean_squared_error(y, y_pred)

print(f'Mean Squared Error: {my_mse}')
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 483925062.1193975
Mean Squared Error: 482034577.4732652


#### Результат схожий